In [7]:
import opendatasets as od
import pandas as pd
import numpy as np

# Eric's Kaggle API key
# "username":"ericthedataguy",
# "key":"875c0de0faea2fefa081c78eb470a347"

od.download(
    "https://www.kaggle.com/datasets/jiashenliu/515k-hotel-reviews-data-in-europe")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:Downloading 515k-hotel-reviews-data-in-europe.zip to ./515k-hotel-reviews-data-in-europe


100%|██████████| 45.1M/45.1M [00:00<00:00, 53.7MB/s]


In [8]:
df = pd.read_csv("515k-hotel-reviews-data-in-europe/Hotel_Reviews.csv")
df.head()

Hotel_Address  \
0   s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   
1   s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   
2   s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   
3   s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   
4   s Gravesandestraat 55 Oost 1092 AA Amsterdam ...   

   Additional_Number_of_Scoring Review_Date  Average_Score   Hotel_Name  \
0                           194    8/3/2017            7.7  Hotel Arena   
1                           194    8/3/2017            7.7  Hotel Arena   
2                           194   7/31/2017            7.7  Hotel Arena   
3                           194   7/31/2017            7.7  Hotel Arena   
4                           194   7/24/2017            7.7  Hotel Arena   

  Reviewer_Nationality                                    Negative_Review  \
0              Russia    I am so angry that i made this post available...   
1             Ireland                                         No Negative   
2           Australia    Rooms are nice but for elderly a bit difficul...   
3      United Kingdom    My room was dirty and I was afraid to walk ba...   
4         New Zealand    You When I booked with your company on line y...   

   Review_Total_Negative_Word_Counts  Total_Number_of_Reviews  \
0                                397                     1403   
1                                  0                     1403   
2                                 42                     1403   
3                                210                     1403   
4                                140                     1403   

                                     Positive_Review  \
0   Only the park outside of the hotel was beauti...   
1   No real complaints the hotel was great great ...   
2   Location was good and staff were ok It is cut...   
3   Great location in nice surroundings the bar a...   
4    Amazing location and building Romantic setting    

   Review_Total_Positive_Word_Counts  \
0                                 11   
1                                105   
2                                 21   
3                                 26   
4                                  8   

   Total_Number_of_Reviews_Reviewer_Has_Given  Reviewer_Score  \
0                                           7             2.9   
1                                           7             7.5   
2                                           9             7.1   
3                                           1             3.8   
4                                           3             6.7   

                                                Tags days_since_review  \
0  [' Leisure trip ', ' Couple ', ' Duplex Double...            0 days   
1  [' Leisure trip ', ' Couple ', ' Duplex Double...            0 days   
2  [' Leisure trip ', ' Family with young childre...            3 days   
3  [' Leisure trip ', ' Solo traveler ', ' Duplex...            3 days   
4  [' Leisure trip ', ' Couple ', ' Suite ', ' St...           10 days   

         lat       lng  
0  52.360576  4.915968  
1  52.360576  4.915968  
2  52.360576  4.915968  
3  52.360576  4.915968  
4  52.360576  4.915968

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Hotel_Address                               515738 non-null  object 
 1   Additional_Number_of_Scoring                515738 non-null  int64  
 2   Review_Date                                 515738 non-null  object 
 3   Average_Score                               515738 non-null  float64
 4   Hotel_Name                                  515738 non-null  object 
 5   Reviewer_Nationality                        515738 non-null  object 
 6   Negative_Review                             515738 non-null  object 
 7   Review_Total_Negative_Word_Counts           515738 non-null  int64  
 8   Total_Number_of_Reviews                     515738 non-null  int64  
 9   Positive_Review                             515738 non-null  object 
 

In [7]:
df['Review_ID'] = df.index


subset = df[['Review_ID', 'Hotel_Name', 'Average_Score', 'Reviewer_Score', 'Tags']]
subset.head()

Review_ID   Hotel_Name  Average_Score  Reviewer_Score  \
0          0  Hotel Arena            7.7             2.9   
1          1  Hotel Arena            7.7             7.5   
2          2  Hotel Arena            7.7             7.1   
3          3  Hotel Arena            7.7             3.8   
4          4  Hotel Arena            7.7             6.7   

                                                Tags  
0  [' Leisure trip ', ' Couple ', ' Duplex Double...  
1  [' Leisure trip ', ' Couple ', ' Duplex Double...  
2  [' Leisure trip ', ' Family with young childre...  
3  [' Leisure trip ', ' Solo traveler ', ' Duplex...  
4  [' Leisure trip ', ' Couple ', ' Suite ', ' St...

In [46]:
subset.loc['Positive'] = (subset['Reviewer_Score'] > subset['Average_Score'])

tags_by_hotel = df.groupby('Hotel_Name')['Tags'].agg(lambda x: set(x)).reset_index()
tags_by_hotel['Tags'] = tags_by_hotel['Tags'].apply(lambda x: ', '.join(x))
tags_by_hotel.head()



Hotel_Name  \
0                  11 Cadogan Gardens   
1                            1K Hotel   
2  25hours Hotel beim MuseumsQuartier   
3                                  41   
4  45 Park Lane Dorchester Collection   

                                                Tags  
0  [' Leisure trip ', ' Couple ', ' Superior Quee...  
1  [' Business trip ', ' Group ', ' Superior M Do...  
2  [' Leisure trip ', ' Solo traveler ', ' Standa...  
3  [' Leisure trip ', ' Couple ', ' Luxury Suite ...  
4  [' Leisure trip ', ' Couple ', ' Superior King...

In [68]:
def hotel_recommendation_system(review_id=0):
    # Get the review details based on the Review_ID
    review = subset.loc[subset['Review_ID'] == review_id]

    # Check if the review is positive
    if review['Positive'].values[0] == 1.0:
        # Get the tags of the current review
        review_tags = review['Tags'].values[0]

        # Find the matching hotel names based on tags using NLTK
        matching_hotels = tags_by_hotel[tags_by_hotel['Tags'].apply(lambda x: len(set(review_tags).intersection(set(x))) >= 2)]

        # Exclude the original hotel name from the matching hotels
        matching_hotels = matching_hotels[matching_hotels['Hotel_Name'] != review['Hotel_Name'].values[0]]

        # Get the top 3 most matching hotel names (if any)
        recommended_hotels = matching_hotels['Hotel_Name'].head(3).tolist()

        # Print the recommendation details
        print("Here's our recommendations for you!")
        print("Original Hotel Name:", review['Hotel_Name'].values[0])
        print("Review Score:", review['Reviewer_Score'].values[0])
        print("Recommended Hotel Names:", recommended_hotels)

    else:
        # Print a message for negative reviews
        print("We are sorry for your dissatisfactory experience.")


In [69]:
hotel_recommendation_system(review_id=7)

Here's our recommendations for you!
Original Hotel Name: Hotel Arena
Review Score: 10.0
Recommended Hotel Names: ['11 Cadogan Gardens', '1K Hotel', '25hours Hotel beim MuseumsQuartier']


In [21]:
subset[subset['Positive'] == 1.0]

Review_ID             Hotel_Name  Average_Score  Reviewer_Score  \
7             7.0            Hotel Arena            7.7            10.0   
9             9.0            Hotel Arena            7.7             7.9   
10           10.0            Hotel Arena            7.7            10.0   
13           13.0            Hotel Arena            7.7             9.2   
14           14.0            Hotel Arena            7.7             8.8   
...           ...                    ...            ...             ...   
515724   515724.0  Atlantis Hotel Vienna            8.1             8.3   
515729   515729.0  Atlantis Hotel Vienna            8.1            10.0   
515732   515732.0  Atlantis Hotel Vienna            8.1            10.0   
515736   515736.0  Atlantis Hotel Vienna            8.1             8.8   
515737   515737.0  Atlantis Hotel Vienna            8.1             8.3   

                                                     Tags Positive  
7       [' Leisure trip ', ' Couple ', ' Duplex Double...      1.0  
9       [' Leisure trip ', ' Couple ', ' Large King Ro...      1.0  
10      [' Leisure trip ', ' Group ', ' Duplex Double ...      1.0  
13      [' Business trip ', ' Solo traveler ', ' Duple...      1.0  
14      [' Leisure trip ', ' Family with young childre...      1.0  
...                                                   ...      ...  
515724  [' Leisure trip ', ' Family with young childre...      1.0  
515729  [' Leisure trip ', ' Couple ', ' Standard Doub...      1.0  
515732  [' Leisure trip ', ' Family with young childre...      1.0  
515736  [' Leisure trip ', ' Group ', ' Standard Tripl...      1.0  
515737  [' Leisure trip ', ' Family with young childre...      1.0  

[302674 rows x 6 columns]